In [ ]:
!pip install langchain_community pypdf chromadb google-generativeai
!pip install -U langchain-google-genai faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:

In [ ]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import Chroma
import pandas as pd
df = pd.read_csv('aboutFiliere.csv', header=None)
df.to_csv('aboutFiliere.csv', index=False)
loader = CSVLoader(
    "aboutFiliere.csv" ,
)
docs = loader.load()
docs[0]

Document(metadata={'source': 'aboutFiliere.csv', 'row': 0}, page_content="0: cp = Cycle Préparatoire  = ap\n1: cordinnateur  du  cycle preparatoire: Pr. Ahmed BOUJRAF\n2: email de cordinnateur  du  cycle preparatoire : a.boujraf@uae.ac.ma\n3: Cycle Préparatoire : Sciences & Techniques pour l’Ingénieur Objectifs de la formation : Préparer et donner à l’élève ingénieur les bases scientifiques et techniques, ainsi que les connaissances en langues et communications indispensables à la poursuite des études dans une filière du cycle ingénieur. Le cursus suivi au cycle préparatoire de l’ENSA d’Al-Hoceima ainsi que celui suivi dans less filières accréditées par l’ENSAH permettent de: Répondre à la demande régionale et nationale en matière de ressources humaines qualifiées dans les domaines du Génie Civil, de l’Environnement, de l'Enérgitique et de l’Informatique. Renforcer la compétitivité et le développement socio-économique régional et national. Compétences à acquérir : Le cycle préparatoire

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
text_splitter.split_documents(docs)[:3]

[Document(metadata={'source': 'aboutFiliere.csv', 'row': 0}, page_content='0: cp = Cycle Préparatoire  = ap\n1: cordinnateur  du  cycle preparatoire: Pr. Ahmed BOUJRAF\n2: email de cordinnateur  du  cycle preparatoire : a.boujraf@uae.ac.ma'),
 Document(metadata={'source': 'aboutFiliere.csv', 'row': 0}, page_content="3: Cycle Préparatoire : Sciences & Techniques pour l’Ingénieur Objectifs de la formation : Préparer et donner à l’élève ingénieur les bases scientifiques et techniques, ainsi que les connaissances en langues et communications indispensables à la poursuite des études dans une filière du cycle ingénieur. Le cursus suivi au cycle préparatoire de l’ENSA d’Al-Hoceima ainsi que celui suivi dans less filières accréditées par l’ENSAH permettent de: Répondre à la demande régionale et nationale en matière de ressources humaines qualifiées dans les domaines du Génie Civil, de l’Environnement, de l'Enérgitique et de l’Informatique. Renforcer la compétitivité et le développement socio-é

In [ ]:
documents= text_splitter.split_documents(docs)

In [ ]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents,GoogleGenerativeAIEmbeddings(model="models/embedding-001"))

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash" , generation_config=genai.GenerationConfig(
        temperature=0.5,
    ))
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x7c82b030fd10>, default_metadata=())

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI Agent specialized in answering questions about ENSAH.
    Use the provided context and chat history to create your response:
    - Refer to both the context and chat history to provide complete answers
    - If a question refers to previous context, use that information
    - If the information cannot be found in context or history, respond with 'thella'
    - Always respond in the same language as the question
    - if the user query

    Context:
    {context}"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])



In [ ]:
#memory = ConversationBufferMemory(
#    memory_key="chat_history",
#    return_messages=True
#)

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

In [ ]:
retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7c82b053bb50>, search_kwargs={})

In [ ]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=chain,
)


response=retrieval_chain.invoke({"input":"quelle  est  le  coridnnateur  de gc?? ??"})
response

{'input': 'quelle  est  le  coridnnateur  de gc?? ??',
 'context': [Document(id='58949bcd-bc63-4a4d-b9ba-68caca0d88f8', metadata={'source': 'aboutFiliere.csv', 'row': 1}, page_content='0: gc = Génie Civil\n1: cordinnateur  du  filiere : Pr. AHMED ZIAN\n2: email de cordinnateur  du  gc a.zian@uae.ac.ma'),
  Document(id='258e88c2-6b1c-4140-a187-5d6371dcfb76', metadata={'source': 'aboutFiliere.csv', 'row': 2}, page_content='0: gi = Génie Informatique\n1: cordinnateur  du  filiere : Pr. IMAD BADI\n2: email de cordinnateur  du  gi :ibadi@uae.ac.ma'),
  Document(id='b4ac3314-e96f-4c8d-b403-d24bd5c942a4', metadata={'source': 'aboutFiliere.csv', 'row': 7}, page_content='0: gm Génie Mécanique\n1: cordinnateur  du  filiere : Pr. khadija Haboubi\n2: email de cordinnateur  du  gm : k.haboubi@uae.ac.ma'),
  Document(id='42c9b822-9d1b-4624-b76f-12c76c7dec35', metadata={'source': 'aboutFiliere.csv', 'row': 0}, page_content='0: cp = Cycle Préparatoire  = ap\n1: cordinnateur  du  cycle preparatoire: Pr

In [ ]:
store = {}

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

conversational_rag_chain = RunnableWithMessageHistory(
    runnable=retrieval_chain,
    get_session_history=get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

In [ ]:
def ask_question(question, session_id="my_session"):
    response = conversational_rag_chain.invoke(
        {"input": question},
        config={"configurable": {"session_id": session_id}}
    )
    return response["answer"]

response1 = ask_question("first : quelle est le  nom comple de addam?")
print(response1)



thella


In [ ]:
response2 = ask_question("c'est quooi cp ")
print(response2)

CP signifie Cycle Préparatoire.


In [ ]:
!pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.4 MB/s eta 0:00:00


In [ ]:
from langchain_experimental.agents import create_csv_agent


In [ ]:
csv_agent = create_csv_agent(
    llm,
    "profsData.csv",
    verbose=True,
    allow_dangerous_code=True
)

In [ ]:
csv_agent

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'df_head': "|    | ensah professor's name   | ensah professor's email   |\n|---:|:-------------------------|:--------------------------|\n|  0 | ABOU EL HANOUNE Younes   | yabouelhanoune@uae.ac.ma  |\n|  1 | ABOUD Miloud             | m.aboud@uae.ac.ma         |\n|  2 | ADDAM Mohamed            | m.addam@uae.ac.ma         |\n|  3 | Akouh Mohamed            | m.akouh@uae.ac.ma         |\n|  4 | ALLAOUZI Imane           | nan                       |", 'tools': 'python_repl_ast - A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.', 'tool_names': 'python_re

In [ ]:
print(csv_agent.agent.runnable.steps[1].template)



You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`:
{df_head}

Begin!
Question: {input}
{agent_scratchpad}


In [ ]:
resp = csv_agent.invoke({"input":"how  many  professors in ensah?", "system" : " if user query about with emails u should neglect the  Nan values , MetaData:{colmn1 :'profs' , colmn2:'emails'} all the  data is about ENSAH "})
print(resp.get("output"))



> Entering new AgentExecutor chain...
Thought: I need to count the number of rows in the dataframe `df` to find out how many professors are listed.
Action: python_repl_ast
Action Input: `print(len(df))`70
Thought: I now know the final answer.
Final Answer: 70

> Finished chain.
70


In [ ]:
from langchain.tools import Tool
from langchain_core.tools.retriever import create_retriever_tool

def query_csv(query: str) -> str:
    return csv_agent.invoke({"input":query, "system" : " if user query about with emails u should neglect the  Nan values , MetaData:{colmn1 :'profs' , colmn2:'emails'} "})

csv_tool = Tool(
    name="csv_tool",
    func=query_csv,
    description=(
        "This tool is for answering questions related to ENSAH professors' names and emails. "
        "It can perform calculations or retrieve specific information from a structured dataframe "
        "with columns for professors and their emails. For email queries, it filters out missing (NaN) values. "
        "If the user asks about emails, ensure you neglect invalid or NaN entries."
    )
)

def query_pdf_tool(query: str) -> str:
    response=retrieval_chain.invoke({"input":query})
    return response["answer"]


pdf_tool = Tool(
    name="pdf_tool",
    func=query_pdf_tool,
    description="this  tool can be  used to  resolve every  question that can't be  responded by  the  other tools , it is the  default tool "
)



In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

prpt  =  ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant he can respond  every  thing  related to  ensah and  professors . and u should choose the  compatible  tool that  can help  user , u shpuld  not ask for  anhy thing  , just  pass the query to  the default  tool ",
        ),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

tools = [csv_tool, pdf_tool]
agent = create_tool_calling_agent(llm, tools, prpt )





In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
response = agent_executor.invoke({"input": "tell me about gc filiere "})
print(response)



> Entering new AgentExecutor chain...

Invoking: `pdf_tool` with `tell me about gc filiere`


La filière GC (Génie Civil) est coordonnée par le Pr. AHMED ZIAN, joignable par email à l'adresse a.zian@uae.ac.ma.

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.contents[2].parts[0].function_response.name: Name cannot be empty.
